In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import WordCloud

st.set_page_config(layout="wide")
st.title("🚗 EVision: AI-Powered Electric Vehicle Insights & Forecasting")

# Load Data
@st.cache_data
def load_data():
    path = r"C:\Users\vijay\Downloads\Electric_Vehicle_Population_Data (3).csv"
    return pd.read_csv(path)

df = load_data()
df['Model Year'] = pd.to_numeric(df['Model Year'], errors='coerce')
df['Electric Range'] = pd.to_numeric(df['Electric Range'], errors='coerce')
df['County'] = df['County'].fillna("Unknown")
df['Make'] = df['Make'].fillna("Unknown")

# Sidebar Filters
st.sidebar.header("🔍 Filter Data")
years = st.sidebar.slider("Select Year Range", int(df['Model Year'].min()), int(df['Model Year'].max()),
                          (int(df['Model Year'].min()), int(df['Model Year'].max())))
brands = st.sidebar.multiselect("Select Brands", options=df['Make'].unique(), default=df['Make'].unique())
counties = st.sidebar.multiselect("Select Counties", options=df['County'].unique(), default=df['County'].unique())

filtered_df = df[
    (df['Model Year'] >= years[0]) & (df['Model Year'] <= years[1]) &
    (df['Make'].isin(brands)) & (df['County'].isin(counties))
]

st.sidebar.download_button("📥 Download Filtered Data", filtered_df.to_csv(index=False),
                           file_name="filtered_ev_data.csv")

# KPIs
st.markdown("### 🔢 Key Metrics")
col1, col2, col3 = st.columns(3)
col1.metric("Total EVs", f"{filtered_df.shape[0]:,}")
col2.metric("Unique Brands", filtered_df['Make'].nunique())
col3.metric("Avg. Range (mi)", f"{filtered_df['Electric Range'].mean():.1f}")

st.markdown("---")

# Chart 1 - EV Growth Over Time
st.subheader("📈 1. EV Growth Over Time")
ev_growth = filtered_df.groupby('Model Year').size().reset_index(name='EV Count')
fig1 = px.line(ev_growth, x='Model Year', y='EV Count', markers=True)
st.plotly_chart(fig1)

# Chart 2 - Word Cloud
st.subheader("💬 2. Simulated Word Cloud of EV Brands")
text = ' '.join(filtered_df['Make'].dropna().astype(str))
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
fig2, ax2 = plt.subplots(figsize=(10, 4))
ax2.imshow(wordcloud, interpolation='bilinear')
ax2.axis('off')
st.pyplot(fig2)

# Chart 3 - KDE Plot of Range
st.subheader("⚡ 3. Electric Range Distribution by Top Brands")
top_brands = filtered_df['Make'].value_counts().nlargest(5).index
fig3, ax3 = plt.subplots()
for brand in top_brands:
    sns.kdeplot(data=filtered_df[filtered_df['Make'] == brand]['Electric Range'], label=brand, fill=True, ax=ax3)
ax3.set_xlabel("Electric Range (mi)")
ax3.set_title("Range Distribution (Top 5 Brands)")
ax3.legend()
st.pyplot(fig3)

# Chart 4 - Forecasting EV Growth (Linear)
st.subheader("🔮 4. Forecasting: Linear Trend Projection")
growth_data = ev_growth.copy()
growth_data['Forecast'] = np.poly1d(np.polyfit(growth_data['Model Year'], growth_data['EV Count'], 1))(growth_data['Model Year'])
fig4 = px.line(growth_data, x='Model Year', y=['EV Count', 'Forecast'], markers=True)
st.plotly_chart(fig4)

# Chart 5 - Charging Infra vs EVs (Simulated)
st.subheader("🔌 5. Correlation: EV Count vs Charging Stations (Simulated)")
charging_df = growth_data.copy()
charging_df['Charging Stations'] = growth_data['EV Count'] * np.random.uniform(0.08, 0.12, size=len(growth_data))
fig5 = px.scatter(charging_df, x='Charging Stations', y='EV Count', size='EV Count', color='Model Year',
                  title="Simulated Charging Infra vs EV Count")
st.plotly_chart(fig5)

# Chart 6 - Policy Incentive Impact
st.subheader("⚖️ 6. Simulated Policy Impact on EV Adoption")
boost = st.slider("Government Incentive Boost %", 0, 100, 20)
policy_df = growth_data.copy()
policy_df['Boosted'] = policy_df['EV Count'] * (1 + boost / 100)
fig6 = px.bar(policy_df, x='Model Year', y=['EV Count', 'Boosted'], barmode='group')
st.plotly_chart(fig6)

# Chart 7 - (Removed due to missing Latitude/Longitude)

# Chart 8 - Grouped Bar by County and Brand
st.subheader("📌 7. EV Count by Make and County")
grouped = filtered_df.groupby(['County', 'Make']).size().reset_index(name='Count')
grouped = grouped[grouped['Make'].isin(top_brands)]
fig8 = px.bar(grouped, x='County', y='Count', color='Make', barmode='group')
st.plotly_chart(fig8)

# Chart 9 - Boxplot
st.subheader("📦 8. Boxplot: Electric Range by Brand")
fig9 = px.box(filtered_df[filtered_df['Make'].isin(top_brands)], x='Make', y='Electric Range')
st.plotly_chart(fig9)

# Chart 10 - Histogram of Range
st.subheader("📊 9. Histogram: Electric Range Distribution")
fig10 = px.histogram(filtered_df, x='Electric Range', nbins=40)
st.plotly_chart(fig10)

# Chart 11 - Animated Bar Race
st.subheader("🏁 10. Animated EV Brand Growth Over Years")
animated_df = filtered_df.groupby(['Model Year', 'Make']).size().reset_index(name='EV Count')
fig11 = px.bar(animated_df, x='Make', y='EV Count', color='Make',
               animation_frame='Model Year', range_y=[0, animated_df['EV Count'].max() + 1000])
st.plotly_chart(fig11)

# Chart 12 - Heatmap: County vs Brand
st.subheader("🧮 11. Heatmap: County vs Brand")
pivot = pd.crosstab(filtered_df['County'], filtered_df['Make'])
fig12, ax12 = plt.subplots(figsize=(14, 6))
sns.heatmap(pivot, cmap="YlGnBu", ax=ax12)
st.pyplot(fig12)

# Chart 13 - Time-Series Forecasting (Rolling Mean)
st.subheader("📈 12. Rolling Avg EV Growth (Time-Series)")
ev_growth['Rolling Avg'] = ev_growth['EV Count'].rolling(window=2, min_periods=1).mean()
fig13 = px.line(ev_growth, x='Model Year', y=['EV Count', 'Rolling Avg'])
st.plotly_chart(fig13)

# Chart 14 - Vehicle Type Share (Stacked Bar)
st.subheader("🚙 13. EV Type Share by Brand (Simulated)")
if 'Electric Vehicle Type' not in filtered_df.columns:
    filtered_df['Electric Vehicle Type'] = np.random.choice(['BEV', 'PHEV', 'FCEV'], size=len(filtered_df))
type_df = filtered_df.groupby(['Make', 'Electric Vehicle Type']).size().reset_index(name='Count')
type_df = type_df[type_df['Make'].isin(top_brands)]
fig14 = px.bar(type_df, x='Make', y='Count', color='Electric Vehicle Type', barmode='stack')
st.plotly_chart(fig14)

# Chart 15 - Price vs Range (Simulated)
st.subheader("💲 14. Simulated EV Price vs Range")
simulated = filtered_df[['Make', 'Electric Range']].copy()
np.random.seed(42)
simulated['Price ($K)'] = simulated['Electric Range'] * 0.4 + np.random.normal(0, 10, size=len(simulated))
fig15 = px.scatter(simulated, x='Electric Range', y='Price ($K)', color='Make')
st.plotly_chart(fig15)

# Footer
st.markdown("---")
st.markdown("🧠 Powered by EVision | An AI-Driven EV Intelligence Dashboard ⚡")